***Downloading Packages***

In [1]:
!pip install -q torch torchvision torchaudio
!pip install -q numpy pandas scikit-learn matplotlib tqdm

***Import packages***

In [2]:
import os
import math
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


***Downloading TSB-UAD***

In [3]:
DATA_ROOT = Path("/content/data")
DATA_ROOT.mkdir(parents=True, exist_ok=True)

ZIP_PATH = DATA_ROOT / "TSB-UAD-Public.zip"

if not ZIP_PATH.exists():
    print("Downloading TSB-UAD-Public.zip ...")
    !wget -q -O /content/data/TSB-UAD-Public.zip "https://www.thedatum.org/datasets/TSB-UAD-Public.zip"
    print("Download done.")

EXTRACTED_FLAG = DATA_ROOT / "tsb_extracted.flag"
if not EXTRACTED_FLAG.exists():
    print("Unzipping ...")
    !unzip -q /content/data/TSB-UAD-Public.zip -d /content/data
    EXTRACTED_FLAG.touch()
    print("Unzip done.")

all_out_files = list(DATA_ROOT.rglob("*.out"))
print("Found", len(all_out_files), ".out files")

sample_path = sorted(all_out_files)[0]
print("Using sample file:", sample_path)


def load_tsb_out_file(path: Path):
    """
    TSB-UAD .out 格式：
    第 1 欄：time series 值
    第 2 欄：label (0: normal, 1: anomaly)
    """
    df = pd.read_csv(path, header=None)
    values = df.iloc[:, 0].astype(float).to_numpy()
    labels = df.iloc[:, 1].astype(int).to_numpy()
    return values, labels


values, labels = load_tsb_out_file(sample_path)
print("Series length:", len(values))
print("Anomaly ratio:", labels.mean())

Download done.
Unzipping ...
Unzip done.
Found 2062 .out files
Using sample file: /content/data/TSB-UAD-Public/Daphnet/S01R02E0.test.csv@1.out
Series length: 28800
Anomaly ratio: 0.05371527777777778


***Sliding Windows***

In [4]:

def sliding_windows(x: np.ndarray, window: int, step: int = 1):
    x = np.asarray(x)
    n = len(x)
    idx = []
    for i in range(0, n - window + 1, step):
        idx.append(x[i:i+window])
    return np.stack(idx, axis=0)

***Map window scores to points***

In [5]:
def map_window_scores_to_points(window_scores: np.ndarray, series_len: int, window: int, step: int = 1):
    window_scores = np.asarray(window_scores)
    n_windows = window_scores.shape[0]
    point_scores = np.zeros(series_len, dtype=float)
    counts = np.zeros(series_len, dtype=int)

    w = window
    idx = 0
    for start in range(0, series_len - w + 1, step):
        s = start
        e = start + w
        point_scores[s:e] += window_scores[idx]
        counts[s:e] += 1
        idx += 1

    counts[counts == 0] = 1
    point_scores /= counts
    return point_scores


***Evaluate***

In [6]:
def evaluate(scores: np.ndarray, labels: np.ndarray):
    scores = np.asarray(scores).ravel()
    labels = np.asarray(labels).ravel().astype(int)
    if len(np.unique(labels)) < 2:
        return {"auc_roc": np.nan, "auc_pr": np.nan}
    auc_roc = roc_auc_score(labels, scores)
    auc_pr = average_precision_score(labels, scores)
    return {"auc_roc": auc_roc, "auc_pr": auc_pr}

***Plot series and scores***

In [7]:

def plot_series_and_scores(values, labels, scores_dict, max_points=2000, title=""):
    """
    values: 1D series
    labels: 0/1
    scores_dict: {name: score_array}
    """
    n = len(values)
    if n > max_points:
        # 太長就截前面 max_points 畫
        values = values[:max_points]
        labels = labels[:max_points]
        scores_dict = {k: v[:max_points] for k, v in scores_dict.items()}
        n = max_points

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

    ax1.plot(values, label="value")
    ax1.scatter(np.where(labels == 1)[0], values[labels == 1],
                marker="x", s=40, label="anomaly (label)")
    ax1.set_ylabel("value")
    ax1.set_title(title)
    ax1.legend(loc="upper right")

    for name, score in scores_dict.items():
        score_norm = (score - score.min()) / (score.max() - score.min() + 1e-8)
        ax2.plot(score_norm, label=name)
    ax2.set_ylabel("normalized score")
    ax2.legend(loc="upper right")

    plt.xlabel("time")
    plt.tight_layout()
    plt.show()

***Series2Graph***

In [8]:
from sklearn.decomposition import PCA
def map_window_scores_to_points(window_scores: np.ndarray, series_len: int, window: int, step: int = 1):
    window_scores = np.asarray(window_scores)
    point_scores = np.zeros(series_len, dtype=float)
    counts = np.zeros(series_len, dtype=int)

    idx = 0
    for start in range(0, series_len - window + 1, step):
        s = start
        e = start + window
        point_scores[s:e] += window_scores[idx]
        counts[s:e] += 1
        idx += 1

    counts[counts == 0] = 1
    point_scores /= counts
    return point_scores

def s2g_subsequence_embedding(values: np.ndarray, ell: int, lam: int):

    vals = np.asarray(values, dtype=float)
    n = len(vals)
    if n < ell or lam <= 0 or lam >= ell:
        return np.zeros((max(1, n - ell + 1), 2), dtype=float)

    subseqs = sliding_windows(vals, window=ell, step=1)
    L_proj = ell - lam + 1
    proj_list = []
    for s in subseqs:
        sm = np.convolve(s, np.ones(lam, dtype=float), mode="valid")

        proj_list.append(sm)
    Proj = np.stack(proj_list, axis=0)
    pca = PCA(n_components=3)
    Proj_r = pca.fit_transform(Proj)
    ry = Proj_r[:, 1]
    rz = Proj_r[:, 2]
    SP_proj = np.stack([ry, rz], axis=1)

    return SP_proj

def s2g_node_creation(SP_proj: np.ndarray,
                      psi_list: np.ndarray,
                      num_bins: int = 30,
                      min_count: int = 3):

    points = np.asarray(SP_proj, dtype=float)
    num_points = points.shape[0]
    if num_points == 0:
        return [], {}

    nodes = []
    node_id = 0

    psi_to_nodes = {}

    for psi_idx, psi in enumerate(psi_list):
        cos_p = np.cos(psi)
        sin_p = np.sin(psi)

        s = points[:, 0] * cos_p + points[:, 1] * sin_p
        s_min, s_max = s.min(), s.max()
        if s_max == s_min:
            continue

        hist, bin_edges = np.histogram(s, bins=num_bins, range=(s_min, s_max))
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2.0
        for b in range(1, num_bins - 1):
            c = hist[b]
            if c < min_count:
                continue
            if c >= hist[b - 1] and c >= hist[b + 1]:
                node = {
                    "id": node_id,
                    "psi_idx": psi_idx,
                    "s_coord": float(bin_centers[b]),
                }
                nodes.append(node)
                psi_to_nodes.setdefault(psi_idx, []).append(node_id)
                node_id += 1

    node_meta = {
        "psi_list": psi_list,
        "psi_to_nodes": psi_to_nodes,
        "nodes": nodes,
    }
    return nodes, node_meta

def s2g_edge_creation(SP_proj: np.ndarray, nodes, node_meta):

    points = np.asarray(SP_proj, dtype=float)
    num_points = points.shape[0]
    if num_points == 0 or len(nodes) == 0:
        return np.zeros(num_points, dtype=int), {}, {}

    psi_list = node_meta["psi_list"]
    psi_to_nodes = node_meta["psi_to_nodes"]

    node_info = {}
    for nd in nodes:
        node_info[nd["id"]] = (nd["psi_idx"], nd["s_coord"])
    thetas = np.arctan2(points[:, 1], points[:, 0])
    thetas = (thetas + 2.0 * np.pi) % (2.0 * np.pi)

    node_seq = np.zeros(num_points, dtype=int)

    for i in range(num_points):
        theta = thetas[i]
        x_y, x_z = points[i, 0], points[i, 1]

        delta = np.abs(psi_list - theta)
        delta = np.minimum(delta, 2.0 * np.pi - delta)
        psi_idx = int(np.argmin(delta))

        if psi_idx not in psi_to_nodes or len(psi_to_nodes[psi_idx]) == 0:
            found = False
            for shift in range(1, len(psi_list)):
                left = (psi_idx - shift) % len(psi_list)
                right = (psi_idx + shift) % len(psi_list)
                if left in psi_to_nodes and len(psi_to_nodes[left]) > 0:
                    psi_idx = left
                    found = True
                    break
                if right in psi_to_nodes and len(psi_to_nodes[right]) > 0:
                    psi_idx = right
                    found = True
                    break
            if not found:
                node_seq[i] = 0
                continue
        cos_p = np.cos(psi_idx * (2.0 * np.pi / len(psi_list)))
        sin_p = np.sin(psi_idx * (2.0 * np.pi / len(psi_list)))
        s_x = x_y * cos_p + x_z * sin_p

        best_node = None
        best_dist = np.inf
        for nid in psi_to_nodes[psi_idx]:
            _, s_coord = node_info[nid]
            d = abs(s_x - s_coord)
            if d < best_dist:
                best_dist = d
                best_node = nid

        node_seq[i] = best_node if best_node is not None else 0

    edge_w = {}
    adj = {}

    for i in range(num_points - 1):
        a = int(node_seq[i])
        b = int(node_seq[i + 1])
        edge_w[(a, b)] = edge_w.get((a, b), 0) + 1

        adj.setdefault(a, set()).add(b)
        adj.setdefault(b, set()).add(a)

    degree = {nid: len(neis) for nid, neis in adj.items()}

    return node_seq, edge_w, degree

def s2g_subsequence_scoring(node_seq: np.ndarray,
                            edge_w: dict,
                            degree: dict,
                            series_len: int,
                            ell: int):
    node_seq = np.asarray(node_seq, dtype=int)
    L = len(node_seq)

    if L <= 1 or series_len <= 0:
        return np.zeros(series_len, dtype=float)

    edge_scores = np.zeros(L - 1, dtype=float)
    for j in range(L - 1):
        a = int(node_seq[j])
        b = int(node_seq[j + 1])
        w_ab = edge_w.get((a, b), 0)
        deg_a = degree.get(a, 0)
        edge_scores[j] = w_ab * max(deg_a - 1, 0)
    point_scores = np.zeros(series_len, dtype=float)
    counts = np.zeros(series_len, dtype=int)

    for j in range(L - 1):
        t_center = j + ell // 2
        if 0 <= t_center < series_len:
            point_scores[t_center] += edge_scores[j]
            counts[t_center] += 1

    counts[counts == 0] = 1
    point_scores /= counts

    anom_scores = -point_scores
    return anom_scores


def series2graph_detector(values: np.ndarray,
                          window: int = 64,
                          step: int = 1,
                          num_angles: int = 64,
                          num_bins: int = 30):
    vals = np.asarray(values, dtype=float)
    n = len(vals)
    if n < window + 2:
        return np.zeros(n, dtype=float)

    lam = max(1, window // 3)
    SP_proj = s2g_subsequence_embedding(vals, ell=window, lam=lam)

    psi_list = np.linspace(0.0, 2.0 * np.pi, num=num_angles, endpoint=False)
    nodes, node_meta = s2g_node_creation(SP_proj, psi_list=psi_list,
                                         num_bins=num_bins, min_count=3)

    if len(nodes) == 0:
        return np.zeros(n, dtype=float)

    node_seq, edge_w, degree = s2g_edge_creation(SP_proj, nodes=nodes, node_meta=node_meta)
##############################  TODO:  #############################################
#Complete the Series2Graph
    point_scores = s2g_subsequence_scoring(
            node_seq=node_seq,
            edge_w=edge_w,
            degree=degree,
            series_len=n,       # 對應總長度 n
            ell=window         # 對應視窗大小 window
        )
########################################################################################
    scaler = MinMaxScaler()
    point_scores = scaler.fit_transform(point_scores.reshape(-1, 1)).ravel()
    return point_scores



***AutoEncoder***

In [9]:
class WindowDataset(Dataset):
    def __init__(self, values: np.ndarray, window: int, step: int = 1):
        self.values = np.asarray(values, dtype=float)
        self.window = window
        self.step = step
        self.windows = sliding_windows(self.values, window=window, step=step)
        self.windows = self.windows.astype(np.float32)

    def __len__(self):
        return self.windows.shape[0]

    def __getitem__(self, idx):
        x = self.windows[idx]
        return torch.from_numpy(x)


class AE_MLP(nn.Module):
    def __init__(self, window: int, hidden_dim: int = 64, bottleneck: int = 16):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(window, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, bottleneck),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(bottleneck, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, window)
        )

    def forward(self, x):
        z = self.encoder(x)
        out = self.decoder(z)
        return out


def autoencoder_detector(values: np.ndarray, window: int = 64, step: int = 1,
                         train_ratio: float = 0.2, epochs: int = 30, batch_size: int = 128,
                         lr: float = 1e-3):

    vals = np.asarray(values, dtype=float)
    dataset_all = WindowDataset(vals, window=window, step=step)
    n_windows = len(dataset_all)
    n_train = max(10, int(train_ratio * n_windows))
    train_dataset = torch.utils.data.Subset(dataset_all, range(n_train))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)

    model = AE_MLP(window=window).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0.0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            recon = model(batch)
            loss = criterion(recon, batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * batch.size(0)
        epoch_loss /= len(train_loader.dataset)
        if (epoch + 1) % 5 == 0:
            print(f"[AE] Epoch {epoch+1}/{epochs} - loss={epoch_loss:.6f}")

    model.eval()
    all_loader = DataLoader(dataset_all, batch_size=batch_size, shuffle=False, drop_last=False)
    window_scores = []
    with torch.no_grad():
        for batch in all_loader:
            batch = batch.to(device)
            recon = model(batch)
            mse = torch.mean((recon - batch) ** 2, dim=1)
            window_scores.append(mse.cpu().numpy())
    window_scores = np.concatenate(window_scores, axis=0)
    point_scores = map_window_scores_to_points(window_scores, len(vals), window=window, step=step)
    scaler = MinMaxScaler()
    point_scores = scaler.fit_transform(point_scores.reshape(-1, 1)).ravel()
    return point_scores

***LSTM-AD***

In [10]:
class WindowSeqDataset(Dataset):
    def __init__(self, values: np.ndarray, window: int, step: int = 1):
        self.values = np.asarray(values, dtype=float)
        self.window = window
        self.step = step
        self.windows = sliding_windows(self.values, window=window, step=step).astype(np.float32)

    def __len__(self):
        return self.windows.shape[0]

    def __getitem__(self, idx):
        x = self.windows[idx]
        x = x.reshape(-1, 1)
        return torch.from_numpy(x)


class LSTMAE(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, latent_dim=32):
        super().__init__()
        self.encoder = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim,
                               num_layers=1, batch_first=True)
        self.enc_fc = nn.Linear(hidden_dim, latent_dim)

        self.decoder = nn.LSTM(input_size=latent_dim, hidden_size=hidden_dim,
                               num_layers=1, batch_first=True)
        self.dec_fc = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        enc_out, (h_n, c_n) = self.encoder(x)
        h_last = enc_out[:, -1, :]
        z = self.enc_fc(h_last)
        z_seq = z.unsqueeze(1).repeat(1, seq_len, 1)

        dec_out, _ = self.decoder(z_seq)
        recon = self.dec_fc(dec_out)
        return recon


def lstm_ad_detector(values: np.ndarray, window: int = 64, step: int = 1,
                     train_ratio: float = 0.2, epochs: int = 30, batch_size: int = 64,
                     lr: float = 1e-3):

    vals = np.asarray(values, dtype=float)
    scaler_vals = MinMaxScaler()
    vals_scaled = scaler_vals.fit_transform(vals.reshape(-1, 1)).ravel()

    dataset_all = WindowSeqDataset(vals_scaled, window=window, step=step)
    n_windows = len(dataset_all)
    n_train = max(10, int(train_ratio * n_windows))
    train_dataset = torch.utils.data.Subset(dataset_all, range(n_train))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)

    model = LSTMAE(input_dim=1, hidden_dim=64, latent_dim=32).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0.0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            recon = model(batch)
            loss = criterion(recon, batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * batch.size(0)
        epoch_loss /= len(train_loader.dataset)
        if (epoch + 1) % 5 == 0:
            print(f"[LSTM-AD] Epoch {epoch+1}/{epochs} - loss={epoch_loss:.6f}")
    model.eval()
    all_loader = DataLoader(dataset_all, batch_size=batch_size, shuffle=False, drop_last=False)
    window_scores = []
    with torch.no_grad():
        for batch in all_loader:
            batch = batch.to(device)
            recon = model(batch)
            mse = torch.mean((recon - batch) ** 2, dim=(1, 2))
            window_scores.append(mse.cpu().numpy())
    window_scores = np.concatenate(window_scores, axis=0)

    point_scores = map_window_scores_to_points(window_scores, len(vals_scaled), window=window, step=step)
    scaler_score = MinMaxScaler()
    point_scores = scaler_score.fit_transform(point_scores.reshape(-1, 1)).ravel()
    return point_scores

***Training***

In [11]:
window_size = 64
step = 1

print("=== Running Series2Graph ===")
scores_s2g = series2graph_detector(values, window=window_size, step=step)
metrics_s2g = evaluate(scores_s2g, labels)
print("Series2Graph-style AUC-ROC = {:.4f}, AUC-PR = {:.4f}".format(
    metrics_s2g["auc_roc"], metrics_s2g["auc_pr"]
))

print("\n=== Running AE-MLP ===")
scores_ae = autoencoder_detector(values, window=window_size, step=step,
                                 train_ratio=0.2, epochs=30, batch_size=128, lr=1e-3)
metrics_ae = evaluate(scores_ae, labels)
print("AE-MLP AUC-ROC = {:.4f}, AUC-PR = {:.4f}".format(
    metrics_ae["auc_roc"], metrics_ae["auc_pr"]
))

print("\n=== Running LSTM-AD (LSTM Autoencoder) ===")
scores_lstm = lstm_ad_detector(values, window=window_size, step=step,
                               train_ratio=0.2, epochs=30, batch_size=64, lr=1e-3)
metrics_lstm = evaluate(scores_lstm, labels)
print("LSTM-AD AUC-ROC = {:.4f}, AUC-PR = {:.4f}".format(
    metrics_lstm["auc_roc"], metrics_lstm["auc_pr"]
))

print("\n=== Summary ===")
print("Series2Graph-style:", metrics_s2g)
print("AE-MLP           :", metrics_ae)
print("LSTM-AD          :", metrics_lstm)

=== Running Series2Graph ===
Series2Graph-style AUC-ROC = 0.6605, AUC-PR = 0.0831

=== Running AE-MLP ===
[AE] Epoch 5/30 - loss=180180.053199
[AE] Epoch 10/30 - loss=149565.944901
[AE] Epoch 15/30 - loss=137121.709265
[AE] Epoch 20/30 - loss=128225.562185
[AE] Epoch 25/30 - loss=121873.575175
[AE] Epoch 30/30 - loss=117500.721613
AE-MLP AUC-ROC = 0.5288, AUC-PR = 0.0609

=== Running LSTM-AD (LSTM Autoencoder) ===
[LSTM-AD] Epoch 5/30 - loss=0.004293
[LSTM-AD] Epoch 10/30 - loss=0.004277
[LSTM-AD] Epoch 15/30 - loss=0.004279
[LSTM-AD] Epoch 20/30 - loss=0.004274
[LSTM-AD] Epoch 25/30 - loss=0.004267
[LSTM-AD] Epoch 30/30 - loss=0.004253
LSTM-AD AUC-ROC = 0.3870, AUC-PR = 0.0407

=== Summary ===
Series2Graph-style: {'auc_roc': np.float64(0.660534837070178), 'auc_pr': np.float64(0.08306349599601073)}
AE-MLP           : {'auc_roc': np.float64(0.5288487006678851), 'auc_pr': np.float64(0.06088029043583067)}
LSTM-AD          : {'auc_roc': np.float64(0.38697062842704655), 'auc_pr': np.float64

### 實驗結果分析

本次作業使用 TSB-UAD 資料集，比較 **Series2Graph** (圖形方法) 與深度學習模型 (**AE-MLP**, **LSTM-AD**) 的異常偵測效能。

#### 1. 效能比較表

| 模型 (Model) | 方法類型 | AUC-ROC | AUC-PR | 排名 |
| :--- | :--- | :--- | :--- | :--- |
| **Series2Graph** | **Graph-based** | **0.6605** | **0.0831** | **1** |
| AE-MLP | Reconstruction | 0.5288 | 0.0609 | 2 |
| LSTM-AD | Forecasting/Recon | 0.3870 | 0.0407 | 3 |

#### 2. 結果觀察

* **Series2Graph 表現最佳**：
    AUC-ROC 達 **0.66**，顯著優於其他模型。這印證了簡報所述，透過圖形節點 (Nodes) 捕捉重複的子序列形狀 (Shape patterns)，能更穩健地辨識出異常路徑，且不依賴大量的訓練參數。

* **深度學習模型表現不佳**：
    * **AE-MLP (0.53)**：僅略高於隨機猜測，這顯示單純利用全連接層 (MLP) 進行壓縮與重建，在捕捉此資料集的時間序列特徵上效果有限，導致正常數據與異常數據的重建誤差差異不夠明顯。
    * **LSTM-AD (0.39)**：表現最差 (AUC < 0.5)，這通常代表模型受到訓練資料中雜訊的影響，又或是產生過擬合 (Overfitting)等情況，導致模型連「異常數據」都能準確重建，使得異常分數無法突顯，無法有效利用重建誤差來偵測異常。

#### 3. 結論
實驗結果符合 Paparrizos 等人 (2025) 的觀察及研究結論：
* **複雜的深度學習模型不一定優於傳統或圖形方法** 。儘管 LSTM 和 AE 是熱門的深度學習方法，但在特定的時間序列基準測試中， Series2Graph 在非監督式且無參數調整的情況下，展現了較佳的魯棒性跟效能。
* **方法特性的差異**： Series2Graph 不需要假設資料分佈，且是非監督式學習，這使其在面對未知型態的異常時更具優勢 ；而重建類方法 (AE, LSTM-AD) 則高度依賴於「正常資料能被良好重建，而異常資料不能」的假設，這在複雜的真實數據中不一定總是成立。